In [6]:
import unittest
import cv2
import argparse
import numpy as np
import time
import closed_form_matting

class TestMatting(unittest.TestCase):
    def test_solution_close_to_original_implementation(self):
        image = cv2.imread('testdata/source.png', cv2.IMREAD_COLOR) / 255.0
        scribles = cv2.imread('testdata/scribbles.png', cv2.IMREAD_COLOR) / 255.0

        alpha = closed_form_matting.closed_form_matting_with_scribbles(image, scribles)
        foreground, background = closed_form_matting.solve_foreground_background(image, alpha)

        matlab_alpha = cv2.imread('testdata/matlab_alpha.png', cv2.IMREAD_GRAYSCALE) / 255.0
        matlab_foreground = cv2.imread('testdata/matlab_foreground.png', cv2.IMREAD_COLOR) / 255.0
        matlab_background = cv2.imread('testdata/matlab_background.png', cv2.IMREAD_COLOR) / 255.0

        sad_alpha = np.mean(np.abs(alpha - matlab_alpha))
        sad_foreground = np.mean(np.abs(foreground - matlab_foreground))
        sad_background = np.mean(np.abs(background - matlab_background))

        self.assertLess(sad_alpha, 1e-2)
        self.assertLess(sad_foreground, 1e-2)
        self.assertLess(sad_background, 1e-2)

    def test_matting_with_trimap(self):
        image = cv2.imread('testdata/source.png', cv2.IMREAD_COLOR) / 255.0
        trimap = cv2.imread('testdata/trimap.png', cv2.IMREAD_GRAYSCALE) / 255.0
        time_start = time.time()
        alpha = closed_form_matting.closed_form_matting_with_trimap(image, trimap)
        print('time cost: ', time.time() - time_start)
        reference_alpha = cv2.imread('testdata/output_alpha.png', cv2.IMREAD_GRAYSCALE) / 255.0

        sad_alpha = np.mean(np.abs(alpha - reference_alpha))
        self.assertLess(sad_alpha, 1e-3)


Hello World


In [ ]:
import os
import re

import setuptools

# Project root directory
root_dir = os.path.dirname(__file__)

# Get version string from __init__.py in the package
with open(os.path.join(root_dir, 'closed_form_matting', '__init__.py')) as f:
    version = re.search(r'__version__ = \'(.*?)\'', f.read()).group(1)

# Get dependency list from requirements.txt
with open(os.path.join(root_dir, 'requirements.txt')) as f:
    requirements = f.read().split()

setuptools.setup(
    name='closed-form-matting',
    version=version,
    author='Marco Forte',
    author_email='fortemarco.irl@gmail.com',
    maintainer='Marco Forte',
    maintainer_email='fortemarco.irl@gmail.com',
    url='https://github.com/MarcoForte/closed-form-matting',
    description='A closed-form solution to natural image matting',
    long_description=open(os.path.join(root_dir, 'README.md')).read(),
    long_description_content_type='text/markdown',
    packages=setuptools.find_packages(),
    classifiers=[
        'Development Status :: 4 - Beta',
        'Intended Audience :: Science/Research',
        'Topic :: Scientific/Engineering :: Image Processing',
        'License :: OSI Approved :: MIT License',
        'Programming Language :: Python :: 3',
    ],
    keywords=['closed-form matting', 'image matting', 'image processing'],
    license='MIT',
    python_requires='>=3.5',
    install_requires=requirements,
    entry_points={
        'console_scripts': [
            'closed-form-matting=closed_form_matting.closed_form_matting:main',
            'solve-foreground-background=closed_form_matting.solve_foreground_background:main',
        ],
    },
)

In [ ]:
#!/usr/bin/env python
"""Setting up closed-form-matting package during pip installation"""

import os
import re

import setuptools

# Project root directory
root_dir = os.path.dirname(__file__)

# Get version string from __init__.py in the package
with open(os.path.join(root_dir, 'closed_form_matting', '__init__.py')) as f:
    version = re.search(r'__version__ = \'(.*?)\'', f.read()).group(1)

# Get dependency list from requirements.txt
with open(os.path.join(root_dir, 'requirements.txt')) as f:
    requirements = f.read().split()

setuptools.setup(
    name='closed-form-matting',
    version=version,
    author='Marco Forte',
    author_email='fortemarco.irl@gmail.com',
    maintainer='Marco Forte',
    maintainer_email='fortemarco.irl@gmail.com',
    url='https://github.com/MarcoForte/closed-form-matting',
    description='A closed-form solution to natural image matting',
    long_description=open(os.path.join(root_dir, 'README.md')).read(),
    long_description_content_type='text/markdown',
    packages=setuptools.find_packages(),
    classifiers=[
        'Development Status :: 4 - Beta',
        'Intended Audience :: Science/Research',
        'Topic :: Scientific/Engineering :: Image Processing',
        'License :: OSI Approved :: MIT License',
        'Programming Language :: Python :: 3',
    ],
    keywords=['closed-form matting', 'image matting', 'image processing'],
    license='MIT',
    python_requires='>=3.5',
    install_requires=requirements,
    entry_points={
        'console_scripts': [
            'closed-form-matting=closed_form_matting.closed_form_matting:main',
            'solve-foreground-background=closed_form_matting.solve_foreground_background:main',
        ],
    },
)


In [ ]:
#!/usr/bin/env python
"""Init script when importing closed-form-matting package"""

from closed_form_matting.closed_form_matting import (
    compute_laplacian,
    closed_form_matting_with_prior,
    closed_form_matting_with_trimap,
    closed_form_matting_with_scribbles,
)
from closed_form_matting.solve_foreground_background import (
    solve_foreground_background
)

__version__ = '1.0.0'
__all__ = [
    'compute_laplacian',
    'closed_form_matting_with_prior',
    'closed_form_matting_with_trimap',
    'closed_form_matting_with_scribbles',
    'solve_foreground_background',
]


In [ ]:
#!/usr/bin/env python 
"""Computes foreground and background images given source image and transparency map.

This module implements foreground and background reconstruction method described in Section 7 of:
    Levin, Anat, Dani Lischinski, and Yair Weiss. "A closed-form solution to natural image
    matting." IEEE Transactions on Pattern Analysis and Machine Intelligence 30.2 (2008): 228-242.

Please note, that the cost-function optimized by this code doesn't perfectly match Eq. 19 of the
paper, since our implementation mimics `solveFB.m` Matlab function provided by the authors of the
original paper (this implementation is
availale at http://people.csail.mit.edu/alevin/matting.tar.gz).

The code can be used in two ways:
    1. By importing solve_foregound_background in your code:
        ```
            from solve_foregound_background import solve_foregound_background
            ...
            foreground, background = solve_foregound_background(image, alpha)
        ```
    2. From command line:
        ```
            ./solve_foregound_background.py image.png alpha.png foreground.png background.png
        ```

Authors: Mikhail Erofeev, Yury Gitman.
"""

import numpy as np
import scipy.sparse
import scipy.sparse.linalg

CONST_ALPHA_MARGIN = 0.02


def __spdiagonal(diag):
    """Produces sparse matrix with given vector on its main diagonal."""
    return scipy.sparse.spdiags(diag, (0,), len(diag), len(diag))


def get_grad_operator(mask):
    """Returns sparse matrix computing horizontal, vertical, and two diagonal gradients."""
    horizontal_left = np.ravel_multi_index(np.nonzero(mask[:, :-1] | mask[:, 1:]), mask.shape)
    horizontal_right = horizontal_left + 1

    vertical_top = np.ravel_multi_index(np.nonzero(mask[:-1, :] | mask[1:, :]), mask.shape)
    vertical_bottom = vertical_top + mask.shape[1]

    diag_main_1 = np.ravel_multi_index(np.nonzero(mask[:-1, :-1] | mask[1:, 1:]), mask.shape)
    diag_main_2 = diag_main_1 + mask.shape[1] + 1

    diag_sub_1 = np.ravel_multi_index(np.nonzero(mask[:-1, 1:] | mask[1:, :-1]), mask.shape) + 1
    diag_sub_2 = diag_sub_1 + mask.shape[1] - 1

    indices = np.stack((
        np.concatenate((horizontal_left, vertical_top, diag_main_1, diag_sub_1)),
        np.concatenate((horizontal_right, vertical_bottom, diag_main_2, diag_sub_2))
    ), axis=-1)
    return scipy.sparse.coo_matrix(
        (np.tile([-1, 1], len(indices)), (np.arange(indices.size) // 2, indices.flatten())),
        shape=(len(indices), mask.size))


def get_const_conditions(image, alpha):
    """Returns sparse diagonal matrix and vector encoding color prior conditions."""
    falpha = alpha.flatten()
    weights = (
        (falpha < CONST_ALPHA_MARGIN) * 100.0 +
        0.03 * (1.0 - falpha) * (falpha < 0.3) +
        0.01 * (falpha > 1.0 - CONST_ALPHA_MARGIN)
    )
    conditions = __spdiagonal(weights)

    mask = falpha < 1.0 - CONST_ALPHA_MARGIN
    right_hand = (weights * mask)[:, np.newaxis] * image.reshape((alpha.size, -1))
    return conditions, right_hand


def solve_foreground_background(image, alpha):
    """Compute foreground and background image given source image and transparency map."""

    consts = (alpha < CONST_ALPHA_MARGIN) | (alpha > 1.0 - CONST_ALPHA_MARGIN)
    grad = get_grad_operator(~consts)
    grad_weights = np.power(np.abs(grad * alpha.flatten()), 0.5)

    grad_only_positive = grad.maximum(0)
    grad_weights_f = grad_weights + 0.003 * grad_only_positive * (1.0 - alpha.flatten())
    grad_weights_b = grad_weights + 0.003 * grad_only_positive * alpha.flatten()

    grad_pad = scipy.sparse.coo_matrix(grad.shape)

    smoothness_conditions = scipy.sparse.vstack((
        scipy.sparse.hstack((__spdiagonal(grad_weights_f) * grad, grad_pad)),
        scipy.sparse.hstack((grad_pad, __spdiagonal(grad_weights_b) * grad))
    ))

    composite_conditions = scipy.sparse.hstack((
        __spdiagonal(alpha.flatten()),
        __spdiagonal(1.0 - alpha.flatten())
    ))

    const_conditions_f, b_const_f = get_const_conditions(image, 1.0 - alpha)
    const_conditions_b, b_const_b = get_const_conditions(image, alpha)

    non_zero_conditions = scipy.sparse.vstack((
        composite_conditions,
        scipy.sparse.hstack((
            const_conditions_f,
            scipy.sparse.coo_matrix(const_conditions_f.shape)
        )),
        scipy.sparse.hstack((
            scipy.sparse.coo_matrix(const_conditions_b.shape),
            const_conditions_b
        ))
    ))

    b_composite = image.reshape(alpha.size, -1)

    right_hand = non_zero_conditions.transpose() * np.concatenate((b_composite,
                                                                   b_const_f,
                                                                   b_const_b))

    conditons = scipy.sparse.vstack((
        non_zero_conditions,
        smoothness_conditions
    ))
    left_hand = conditons.transpose() * conditons

    solution = scipy.sparse.linalg.spsolve(left_hand, right_hand).reshape(2, *image.shape)
    foreground = solution[0, :, :, :].reshape(*image.shape)
    background = solution[1, :, :, :].reshape(*image.shape)
    return foreground, background


In [ ]:
#!/usr/bin/env python 
"""Implementation of Closed-Form Matting.

This module implements natural image matting method described in:
    Levin, Anat, Dani Lischinski, and Yair Weiss. "A closed-form solution to natural image matting."
    IEEE Transactions on Pattern Analysis and Machine Intelligence 30.2 (2008): 228-242.

The code can be used in two ways:
    1. By importing solve_foregound_background in your code:
        ```
            import closed_form_matting
            ...
            # For scribles input
            alpha = closed_form_matting.closed_form_matting_with_scribbles(image, scribbles)

            # For trimap input
            alpha = closed_form_matting.closed_form_matting_with_trimap(image, trimap)

            # For prior with confidence
            alpha = closed_form_matting.closed_form_matting_with_prior(
                image, prior, prior_confidence, optional_const_mask)

            # To get Matting Laplacian for image
            laplacian = compute_laplacian(image, optional_const_mask)
        ```
    2. From command line:
        ```
            # Scribbles input
            ./closed_form_matting.py input_image.png -s scribbles_image.png  -o output_alpha.png

            # Trimap input
            ./closed_form_matting.py input_image.png -t scribbles_image.png  -o output_alpha.png

            # Add flag --solve-fg to compute foreground color and output RGBA image instead
            # of alpha.
        ```
"""

from __future__ import division

import logging

import cv2
import numpy as np
from numpy.lib.stride_tricks import as_strided
import scipy.sparse
import scipy.sparse.linalg

def _rolling_block(A, block=(3, 3)):
    """Applies sliding window to given matrix."""
    shape = (A.shape[0] - block[0] + 1, A.shape[1] - block[1] + 1) + block
    strides = (A.strides[0], A.strides[1]) + A.strides
    return as_strided(A, shape=shape, strides=strides)


def compute_laplacian(img: np.ndarray, mask=None, eps: float =10**(-7), win_rad: int =1):
    """Computes Matting Laplacian for a given image.

    Args:
        img: 3-dim numpy matrix with input image
        mask: mask of pixels for which Laplacian will be computed.
            If not set Laplacian will be computed for all pixels.
        eps: regularization parameter controlling alpha smoothness
            from Eq. 12 of the original paper. Defaults to 1e-7.
        win_rad: radius of window used to build Matting Laplacian (i.e.
            radius of omega_k in Eq. 12).
    Returns: sparse matrix holding Matting Laplacian.
    """

    win_size = (win_rad * 2 + 1) ** 2
    h, w, d = img.shape
    # Number of window centre indices in h, w axes
    c_h, c_w = h - 2 * win_rad, w - 2 * win_rad
    win_diam = win_rad * 2 + 1

    indsM = np.arange(h * w).reshape((h, w))
    ravelImg = img.reshape(h * w, d)
    win_inds = _rolling_block(indsM, block=(win_diam, win_diam))

    win_inds = win_inds.reshape(c_h, c_w, win_size)
    if mask is not None:
        mask = cv2.dilate(
            mask.astype(np.uint8),
            np.ones((win_diam, win_diam), np.uint8)
        ).astype(bool)
        win_mask = np.sum(mask.ravel()[win_inds], axis=2)
        win_inds = win_inds[win_mask > 0, :]
    else:
        win_inds = win_inds.reshape(-1, win_size)

    
    winI = ravelImg[win_inds]

    win_mu = np.mean(winI, axis=1, keepdims=True)
    win_var = np.einsum('...ji,...jk ->...ik', winI, winI) / win_size - np.einsum('...ji,...jk ->...ik', win_mu, win_mu)

    A = win_var + (eps/win_size)*np.eye(3)
    B = (winI - win_mu).transpose(0, 2, 1)
    X = np.linalg.solve(A, B).transpose(0, 2, 1)
    vals = np.eye(win_size) - (1.0/win_size)*(1 + X @ B)

    nz_indsCol = np.tile(win_inds, win_size).ravel()
    nz_indsRow = np.repeat(win_inds, win_size).ravel()
    nz_indsVal = vals.ravel()
    L = scipy.sparse.coo_matrix((nz_indsVal, (nz_indsRow, nz_indsCol)), shape=(h*w, h*w))

    # rewrite L in CSR format
    L = scipy.sparse.csr_matrix((nz_indsVal, nz_indsCol, np.arange(0, nz_indsVal.shape[0] + 1, win_size)), shape=(h*w, h*w))
    return L


def closed_form_matting_with_prior(image, prior, prior_confidence, consts_map=None):
    """Applies closed form matting with prior alpha map to image.

    Args:
        image: 3-dim numpy matrix with input image.
        prior: matrix of same width and height as input image holding apriori alpha map.
        prior_confidence: matrix of the same shape as prior hodling confidence of prior alpha.
        consts_map: binary mask of pixels that aren't expected to change due to high
            prior confidence.

    Returns: 2-dim matrix holding computed alpha map.
    """

    assert image.shape[:2] == prior.shape, ('prior must be 2D matrix with height and width equal '
                                            'to image.')
    assert image.shape[:2] == prior_confidence.shape, ('prior_confidence must be 2D matrix with '
                                                       'height and width equal to image.')
    assert (consts_map is None) or image.shape[:2] == consts_map.shape, (
        'consts_map must be 2D matrix with height and width equal to image.')

    logging.info('Computing Matting Laplacian.')
    laplacian = compute_laplacian(image, ~consts_map if consts_map is not None else None)
    confidence = scipy.sparse.diags(prior_confidence.ravel())
    logging.info('Solving for alpha.')
    solution = scipy.sparse.linalg.spsolve(
        laplacian + confidence,
        prior.ravel() * prior_confidence.ravel()
    )
    alpha = np.minimum(np.maximum(solution.reshape(prior.shape), 0), 1)
    return alpha


def closed_form_matting_with_trimap(image, trimap, trimap_confidence=100.0):
    """Apply Closed-Form matting to given image using trimap."""

    assert image.shape[:2] == trimap.shape, ('trimap must be 2D matrix with height and width equal '
                                             'to image.')
    consts_map = (trimap < 0.1) | (trimap > 0.9)
    return closed_form_matting_with_prior(image, trimap, trimap_confidence * consts_map, consts_map)


def closed_form_matting_with_scribbles(image, scribbles, scribbles_confidence=100.0):
    """Apply Closed-Form matting to given image using scribbles image."""

    assert image.shape == scribbles.shape, 'scribbles must have exactly same shape as image.'
    prior = np.sign(np.sum(scribbles - image, axis=2)) / 2 + 0.5
    consts_map = prior != 0.5
    return closed_form_matting_with_prior(
        image,
        prior,
        scribbles_confidence * consts_map,
        consts_map
    )


closed_form_matting = closed_form_matting_with_trimap

def main():
    import argparse

    logging.basicConfig(level=logging.INFO)
    arg_parser = argparse.ArgumentParser(description=__doc__)
    arg_parser.add_argument('image', type=str, help='input image')

    arg_parser.add_argument('-t', '--trimap', type=str, help='input trimap')
    arg_parser.add_argument('-s', '--scribbles', type=str, help='input scribbles')
    arg_parser.add_argument('-o', '--output', type=str, required=True, help='output image')
    arg_parser.add_argument(
        '--solve-fg', dest='solve_fg', action='store_true',
        help='compute foreground color and output RGBA image'
    )
    args = arg_parser.parse_args()

    image = cv2.imread(args.image, cv2.IMREAD_COLOR) / 255.0

    if args.scribbles:
        scribbles = cv2.imread(args.scribbles, cv2.IMREAD_COLOR) / 255.0
        alpha = closed_form_matting_with_scribbles(image, scribbles)
    elif args.trimap:
        trimap = cv2.imread(args.trimap, cv2.IMREAD_GRAYSCALE) / 255.0
        alpha = closed_form_matting_with_trimap(image, trimap)
    else:
        logging.error('Either trimap or scribbles must be specified.')
        arg_parser.print_help()
        exit(-1)

    if args.solve_fg:
        from closed_form_matting.solve_foreground_background import solve_foreground_background
        foreground, _ = solve_foreground_background(image, alpha)
        output = np.dstack((foreground, alpha))
    else:
        output = alpha

    cv2.imwrite(args.output, output * 255.0)


if __name__ == "__main__":
    main()
